**Data cleaning & processing**

source : https://www.kaggle.com/code/rohitgarud/all-almost-data-preprocessing-techniques-for-nlp

In [1]:
!pip install contractions 
!pip install symspellpy
!pip install nltk
!pip install symspellpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp310-cp310-linux_x86_64.whl size=47739 sha256=dc824f2fb2d8b943300917c9d4758bf043f212589f005a136951a774dedaef78
  Stored in directory: /root/.cache/pip/wheels/88/6a/a6/a1283cc145323a1fb3d475bd158ee60b248ab1985230d266fc
Successfully built editdistpy


In [2]:
import re
from nltk.corpus import stopwords
import string
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import contractions
from unidecode import unidecode
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pkg_resources
from symspellpy import SymSpell, Verbosity
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pkg_resources
from symspellpy import SymSpell, Verbosity
import requests

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train["keyword"]= train["keyword"].fillna("")
train["tweet"] = train["keyword"]+ " " + train["text"]
train.sample(5, random_state= 5)

,id,keyword,location,text,target,tweet
6571,9404,survivors,"Chicago, IL",RT @kotowsa: South SudanÛªs war on women: sur...,1,survivors RT @kotowsa: South SudanÛªs war on ...
4830,6877,mass%20murder,Everywhere,This Attempted Mass Murder brought to You by t...,0,mass%20murder This Attempted Mass Murder broug...
2019,2898,damage,Your Conversation,This real shit will damage a bitch,0,damage This real shit will damage a bitch
7427,10625,wounded,NaN,Officer wounded suspect killed in exchange of ...,1,wounded Officer wounded suspect killed in exch...
4983,7109,military,NaN,Hat #russian soviet army kgb military #cossac...,0,military Hat #russian soviet army kgb militar...


In [5]:
train["tweet_lowercase"]= train["tweet"].str.lower()
train["tweet_lowercase"].sample(5, random_state= 5)

6571    survivors rt @kotowsa: south sudanûªs war on ...
4830    mass%20murder this attempted mass murder broug...
2019            damage this real shit will damage a bitch
7427    wounded officer wounded suspect killed in exch...
4983    military hat #russian soviet army kgb  militar...
Name: tweet_lowercase, dtype: object

In [6]:
def remove_html(text):
    soup = BeautifulSoup(text)
    text = soup.get_text()
    return text
def remove_urls(text):
    pattern = re.compile(r'https?://(www\.)?(\w+)(\.\w+)(/\w*)?')
    text= re.sub(pattern, "", text)
    return(text)
def remove_email(text):
    pattern= re.compile(r"[\w\.-]+@[\w\.-]+\.\w+")
    text = re.sub(pattern, "", text)
    return text
def removementions (text):
    pattern = re.compile(r"@\w+")
    text = re.sub(pattern, "", text)
    return text
def handle_accents(text) :
    text = unidecode(text)
    return(text)
def remove_unicode_chars(text):
    text = text.encode("ascii", "ignore").decode()
    return text
def remove_punctuations(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), " ",text)
    return text
punct_to_keep = ["&"]
punct_to_remove = "".join(punct for punct in string.punctuation if punct not in punct_to_keep)
def handle_punctuations(text):
    text = re.sub('[%s]' % re.escape(punct_to_remove), " ",text)
    for punct in punct_to_keep:
        text = re.sub(f"{punct}", f" {punct} ",text)
    return text
def normalize_abbreviations(text):
    matches = re.finditer(r"([A-Z]\.)+", text)
    matched_abbr = [match.group() for match in matches]
    for abbr in matched_abbr:
        text = re.sub(abbr,abbr.replace(".",""), text)
    return text
def handle_amount_and_percentage(text):
    text = re.sub(r"(₹|\$|£|€|¥)\s?\d+(\.\d+)?", "money amount",text)
    text = re.sub(r"\d+(\.\d+)?\s?%", "percentage",text)
    return text
def remove_digits(text):
    pattern = re.compile("\w*\d+\w*")
    text = re.sub(pattern, "",text)
    return text

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])
def remove_extra_spaces(text):
    text = re.sub(' +', ' ', text).strip()
    return text

In [7]:
###removing html
train["tweet_nohtml"]=train["tweet_lowercase"].apply(remove_html)
train["tweet_nohtml"].sample(5, random_state= 5)

###removing contractions
train["tweetnocontracrions"] = train["tweet_nohtml"].apply(contractions.fix)
train["tweetnocontracrions"].sample(5, random_state=5)

###removing urls
train["nourl"]=train["tweetnocontracrions"].apply(remove_urls)
train["nourl"].sample(5, random_state=5)

###removing email
train["noemail"]=train["nourl"].apply(remove_email)
train["noemail"].sample(5, random_state= 5)

###removing mentions 
train["nomentions"]=train["nourl"].apply(removementions)
train["nomentions"].sample(5, random_state= 5)

###removing accent 
train["noaccent"]=train["nomentions"].apply(handle_accents)
train["noaccent"].sample(5, random_state= 5)

###removing unicode 
train["nounicode"]=train["noaccent"].apply(remove_unicode_chars)
train["nounicode"].sample(5, random_state= 5)


###removing punctuations 
train["nopuncts"] = train["nounicode"].apply(remove_punctuations)
train["nopuncts"].sample(5, random_state=5)

###removing digits 
train["nodigits"] = train["nopuncts"].apply(remove_digits)
train["nodigits"].sample(5, random_state=5)

###removing stopwords 
stop_words = set(stopwords.words('english'))
train["nostopwords"] = train["nodigits"].apply(remove_stopwords)
train["nostopwords"].sample(5, random_state=5)

###removing extraspaces 
train["noextraspaces"] = train["nostopwords"].apply(remove_extra_spaces)
train["noextraspaces"].sample(5, random_state=5)

/tmp/ipykernel_28/1048548185.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text)


6571    survivors rt south sudanuas war women survivor...
4830    mass attempted mass murder brought obama admin...
2019                        damage real shit damage bitch
7427    wounded officer wounded suspect killed exchang...
4983    military hat russian soviet army kgb military ...
Name: noextraspaces, dtype: object

In [8]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
print(lemmatizer)


<WordNetLemmatizer>


In [9]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/


Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [10]:
def lemmatize_text(text):
    words = [lemmatizer.lemmatize(word) for word in text.split()]
    text = ' '.join(words)
    return text

In [11]:
train["lemmatised"] = train["noextraspaces"].apply(lemmatize_text)
train["lemmatised"].sample(5, random_state=10)

6524    survive cannot survive without referring witho...
701                                       blazing blazing
3119    electrocuted baby actually get electrocuted wa...
4204    hazard precious cargo onesie recalled choking ...
1651    collapsed portable closet collapsed finally br...
Name: lemmatised, dtype: object

In [12]:
train["lemmatised"] = train["noextraspaces"].apply(lemmatize_text)
train["lemmatised"].sample(5, random_state=10)

6524    survive cannot survive without referring witho...
701                                       blazing blazing
3119    electrocuted baby actually get electrocuted wa...
4204    hazard precious cargo onesie recalled choking ...
1651    collapsed portable closet collapsed finally br...
Name: lemmatised, dtype: object

In [13]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt"
)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [14]:
def correct_spelling_symspell(text):
    words = [
        sym_spell.lookup(
            word, 
            Verbosity.CLOSEST, 
            max_edit_distance=2,
            include_unknown=True
            )[0].term 
        for word in text.split()] 
    text = " ".join(words)
    return text

In [15]:
train["spellcheck"] = train["lemmatised"].apply(correct_spelling_symspell)
train["spellcheck"].sample(5, random_state=10)

6524    survive cannot survive without referring witho...
701                                       blazing blazing
3119    electrocuted baby actually get electrocuted wa...
4204    hazard precious cargo ones recalled choking ha...
1651    collapsed portable closet collapsed finally br...
Name: spellcheck, dtype: object

In [16]:
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt"
)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

True

In [17]:
def correct_spelling_symspell_compound(text):
    words = [
        sym_spell.lookup_compound(
            word, 
            max_edit_distance=2
            )[0].term 
        for word in text.split()] 
    text = " ".join(words)
    return text

In [18]:
train["spellcheck_compound"] = train["spellcheck"].apply(correct_spelling_symspell_compound)
train["spellcheck_compound"].sample(5, random_state=10)

6524    survive cannot survive without referring witho...
701                                       blazing blazing
3119    electrocuted baby actually get electrocuted wa...
4204    hazard precious cargo ones recalled choking ha...
1651    collapsed portable closet collapsed finally br...
Name: spellcheck_compound, dtype: object

In [19]:
url ="https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/american_spellings.json"
american_to_british_dict = requests.get(url).json()

url ="https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
british_to_american_dict = requests.get(url).json()

In [20]:
def britishize(text):
    text = [american_to_british_dict[word] if word in american_to_british_dict else word for word in text.split()]
    return " ".join(text)


def americanize(text):
    text = [british_to_american_dict[word] if word in british_to_american_dict else word for word in text.split()]   
    return " ".join(text)

In [21]:
train["american"] = train["spellcheck_compound"].apply(americanize)
train["american"].sample(5, random_state=10)

6524    survive cannot survive without referring witho...
701                                       blazing blazing
3119    electrocuted baby actually get electrocuted wa...
4204    hazard precious cargo ones recalled choking ha...
1651    collapsed portable closet collapsed finally br...
Name: american, dtype: object

In [22]:
train["british"] = train["spellcheck_compound"].apply(britishize)
train["british"].sample(5, random_state=10)

6524    survive cannot survive without referring witho...
701                                       blazing blazing
3119    electrocuted baby actually get electrocuted wa...
4204    hazard precious cargo ones recalled choking ha...
1651    collapsed portable closet collapsed finally br...
Name: british, dtype: object

In [23]:
train["final"] = train["spellcheck_compound"].apply(remove_stopwords)
train["final"].sample(5, random_state=10)

6524    survive cannot survive without referring witho...
701                                       blazing blazing
3119    electrocuted baby actually get electrocuted wa...
4204    hazard precious cargo ones recalled choking ha...
1651    collapsed portable closet collapsed finally br...
Name: final, dtype: object

In [24]:
train.to_csv("distaster_tweets_cleaned.csv")

# **second part **

In [25]:
!pip install tensorflow
!pip install transformers
!pip install tokenizers

In [26]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

In [27]:
train_data = pd.read_csv('/kaggle/working/distaster_tweets_cleaned.csv')  # Replace with the path to your training data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [28]:
input_ids = []
attention_masks = []
for text in train_data['text']:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='tf',
        truncation=True
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

train_inputs = tf.concat(input_ids, axis=0)
train_masks = tf.concat(attention_masks, axis=0)
train_labels = tf.convert_to_tensor(train_data['target'], dtype=tf.int64)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the model
model.fit(
    [train_inputs, train_masks],
    train_labels,
    epochs=10,
    batch_size=64
)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
119/119 [==============================] - 227s 1s/step - loss: 0.4648 - accuracy: 0.7877
Epoch 2/10
119/119 [==============================] - 176s 1s/step - loss: 0.3380 - accuracy: 0.8667
Epoch 3/10
119/119 [==============================] - 176s 1s/step - loss: 0.2745 - accuracy: 0.8968
Epoch 4/10
119/119 [==============================] - 176s 1s/step - loss: 0.2011 - accuracy: 0.9271
Epoch 5/10
119/119 [==============================] - 176s 1s/step - loss: 0.1628 - accuracy: 0.9448
Epoch 6/10
119/119 [==============================] - 176s 1s/step - loss: 0.1265 - accuracy: 0.9552
Epoch 7/10


In [ ]:

merged_df = submission.merge(test_data, on='id', how='left')
print(merged_df)
merged_df.to_csv("test with target.csv")


In [35]:
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')  # Replace with the path to your test data

# Tokenize and preprocess test data similarly to training data
# ...

# Make predictions on test data
test_inputs = tf.concat(test_input_ids, axis=0)
test_masks = tf.concat(test_attention_masks, axis=0)

predictions = model.predict([test_inputs, test_masks])


NameError: name 'test_input_ids' is not defined

you can evaluate yor test by combining the target col in submission.csv with the test data in test.csv

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Assuming you have true labels for the test data in a variable 'true_labels'
true_labels = [0, 1, 0, 1, 0, 1, 0, 0]  # Replace with your actual test labels

# Assuming 'predictions' contains the model's predictions for the test data
# predictions = [0, 1, 0, 1, 0, 1, 0, 0]  # Replace with your model's predictions

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
roc_auc = roc_auc_score(true_labels, predictions)

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC AUC: {roc_auc:.2f}')
